### 预备：申明模块

使用前需要定义下述变量：

gamma--衰减率

gridsize--世界的行列数

goal--目标坐标

bad--惩罚区域坐标

In [18]:
import numpy as np
import copy

# 创建坐标点类
class Agrid:
    def __init__(self,x=0,y=0):
        self.x=x  # 第几行
        self.y=y  # 第几列
        self.acts=self.create_acts() # 可以采取的行动
        self.rewards=self.create_rewards() # 上述行动对应的回报
        self.v=0  # VI：初始化值函数为0
        self.pi=0,0 # PI: 初始化策略为原地不动

    def create_acts(self):
        acts=[(-1,0),(0,-1),(1,0),(0,1),(0,0)]  # 对应↑ ← ↓ → o
        # 根据约束去除
        if self.x==0: acts.remove((-1,0))
        if self.y==0: acts.remove((0,-1))
        if self.x+1==gridsize[0]: acts.remove((1,0))
        if self.y+1==gridsize[1]: acts.remove((0,1))
        if self.x==goal[0] and self.y==goal[1]: acts=[(0,0)]
        return acts
    
    def create_rewards(self):
        answer=[]
        for a in self.acts:
            # 不动就没有回报
            if a==(0,0): 
                answer.append(0) 
            # 遇到障碍
            elif (self.x+a[0],self.y+a[1]) in bad: 
                answer.append(-1)
            # 到达目标
            elif (self.x+a[0],self.y+a[1])==goal: 
                answer.append(1)
            # 其他情况
            else: 
                answer.append(0)  
        return answer

# 值提升
def VI(grid=[[Agrid(i,j) for j in range(gridsize[1])] for i in range(gridsize[0])]):
    eps = 1e-4
    while True:
        last_grid=copy.deepcopy(grid)
        error=0
        for i in range(gridsize[0]):
            for j in range(gridsize[1]):
                value=last_grid[i][j].v
                for a in grid[i][j].acts:
                    index=grid[i][j].acts.index(a)
                    # 值迭代公式
                    now_v=grid[i][j].rewards[index]+ gamma*last_grid[i+a[0]][j+a[1]].v
                    # print(now_v)
                    if now_v>value:
                        value=now_v
                        grid[i][j].pi=a
                grid[i][j].v=value
                error+=value-last_grid[i][j].v
        if error<eps: 
            return grid

# 策略提升
def PI(grid=[[Agrid(i,j) for j in range(gridsize[1])] for i in range(gridsize[0])]):
    eps = 1e-4
    while True:
        last_grid_for_pi=copy.deepcopy(grid)
        flag=False   # 判断策略提升一步前后有无策略变化
        # 策略评估
        while True:
            last_grid=copy.deepcopy(grid)
            error=0
            for i in range(gridsize[0]):
                for j in range(gridsize[1]):
                    pi=grid[i][j].pi
                    index=grid[i][j].acts.index(pi)
                    # 迭代公式
                    grid[i][j].v=grid[i][j].rewards[index]+\
                                    gamma*last_grid[i+pi[0]][j+pi[1]].v 
                    error+=abs(grid[i][j].v-last_grid[i][j].v)
            if error<=eps: break

        # 策略提升
        for i in range(gridsize[0]):
            for j in range(gridsize[1]):
                value=grid[i][j].v
                # 遍历所有可能动作取最大
                for a in grid[i][j].acts:
                    index=grid[i][j].acts.index(a)
                    # 值迭代公式
                    now_v=grid[i][j].rewards[index]+ gamma*last_grid[i+a[0]][j+a[1]].v
                    if now_v>value:
                        value=now_v
                        grid[i][j].pi=a
                # 判断策略是否改变
                if last_grid_for_pi[i][j].pi!=grid[i][j].pi: flag=True
        if flag==False: 
            return grid

### $\gamma$ =0.9 

In [17]:
gamma=0.9
gridsize=4,4     # 行，列
goal=2,2
bad=[(1,2),(2,1),(3,1)]

值迭代

In [3]:
print('值迭代部分')
print('状态值函数：')
answer=VI()
for i in range(gridsize[0]):
    for j in range(gridsize[1]):
        print("{:.3f}".format(answer[i][j].v),end=' ')
    print('\n')
print('策略：')
acts=[(-1,0),(0,-1),(1,0),(0,1),(0,0)]
vis=['↑', '←', '↓', '→', 'o']
for i in range(gridsize[0]):
    for j in range(gridsize[1]):
        print(vis[acts.index(answer[i][j].pi)],end=' ')
    print('\n')

值迭代部分
状态值函数：
0.590 0.656 0.729 0.810 

0.531 0.590 1.000 0.900 

0.478 1.000 0.000 1.000 

0.430 0.900 1.000 0.900 

策略：
→ → → ↓ 

↑ ↑ ↓ ↓ 

↑ → o ← 

↑ → ↑ ↑ 



策略迭代

In [16]:
print('策略迭代部分')
print('状态值函数：')
answer=PI()
for i in range(gridsize[0]):
    for j in range(gridsize[1]):
        print("{:.3f}".format(answer[i][j].v),end=' ')
    print('\n')
print('策略：')
acts=[(-1,0),(0,-1),(1,0),(0,1),(0,0)]
vis=['↑', '←', '↓', '→', 'o']
for i in range(gridsize[0]):
    for j in range(gridsize[1]):
        print(vis[acts.index(answer[i][j].pi)],end=' ')
    print('\n')

策略迭代部分
状态值函数：
0.590 0.656 0.729 0.810 

0.531 0.590 1.000 0.900 

0.478 1.000 0.000 1.000 

0.430 0.900 1.000 0.900 

策略：
→ → → ↓ 

↑ ↑ ↓ ↓ 

↑ → o ← 

↑ → ↑ ↑ 



### $\gamma$=0.5

In [19]:
gamma=0.5
gridsize=4,4     # 行，列
goal=2,2
bad=[(1,2),(2,1),(3,1)]

值迭代

In [20]:
print('值迭代部分')
print('状态值函数：')
answer=VI()
for i in range(gridsize[0]):
    for j in range(gridsize[1]):
        print("{:.3f}".format(answer[i][j].v),end=' ')
    print('\n')
print('策略：')
acts=[(-1,0),(0,-1),(1,0),(0,1),(0,0)]
vis=['↑', '←', '↓', '→', 'o']
for i in range(gridsize[0]):
    for j in range(gridsize[1]):
        print(vis[acts.index(answer[i][j].pi)],end=' ')
    print('\n')

值迭代部分
状态值函数：
0.031 0.062 0.125 0.250 

0.016 0.031 1.000 0.500 

0.008 1.000 0.000 1.000 

0.004 0.500 1.000 0.500 

策略：
→ → → ↓ 

↑ ↑ ↓ ↓ 

↑ → o ← 

↑ → ↑ ↑ 



策略迭代

In [21]:
print('策略迭代部分')
print('状态值函数：')
answer=PI()
for i in range(gridsize[0]):
    for j in range(gridsize[1]):
        print("{:.3f}".format(answer[i][j].v),end=' ')
    print('\n')
print('策略：')
acts=[(-1,0),(0,-1),(1,0),(0,1),(0,0)]
vis=['↑', '←', '↓', '→', 'o']
for i in range(gridsize[0]):
    for j in range(gridsize[1]):
        print(vis[acts.index(answer[i][j].pi)],end=' ')
    print('\n')

策略迭代部分
状态值函数：
0.031 0.062 0.125 0.250 

0.016 0.031 1.000 0.500 

0.008 1.000 0.000 1.000 

0.004 0.500 1.000 0.500 

策略：
→ → → ↓ 

↑ ↑ ↓ ↓ 

↑ → o ← 

↑ → ↑ ↑ 



观察发现，$\gamma=0.9$ 和 $\gamma=0.5$ 最终的最优策略是一样的，都是会绕过障碍。

我认为这是因为本题中的模型比较简单，不碰到障碍的行为回报为0，也即无代价；而目标又只有一个，也即一条路径最多只有1个正回报。因此在提取策略时不经过障碍的路径收益一定高于经过障碍的，策略就表现为不采取经过障碍的行动。